In [66]:
import json
import re
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix

In [23]:
readJSON=open("News_Category_Dataset_v2.json", "r", encoding="utf-8")
arrayJSON=readJSON.readlines()
writer= open("categories.json","w+") #Escribir archivo
writer.write("[")
for strJSON in arrayJSON:
    dataStore = json.loads(strJSON)
    if (dataStore["category"]== 'ENTERTAINMENT' or dataStore["category"]=='POLITICS' or dataStore["category"]=='TECH' ):
        writer.write(json.dumps(dataStore)+",")
writer.write("{}]")
writer.close()

In [26]:
readJSON=open("categories.json", "r", encoding="utf-8")
arrayCategories=json.loads(readJSON.read())
del arrayCategories[len(arrayCategories)-1] #Eliminar el vacio

In [49]:
df = pd.DataFrame(arrayCategories)
df

category                                           headline  \
0      ENTERTAINMENT  Will Smith Joins Diplo And Nicky Jam For The 2...   
1      ENTERTAINMENT    Hugh Grant Marries For The First Time At Age 57   
2      ENTERTAINMENT  Jim Carrey Blasts 'Castrato' Adam Schiff And D...   
3      ENTERTAINMENT  Julianna Margulies Uses Donald Trump Poop Bags...   
4      ENTERTAINMENT  Morgan Freeman 'Devastated' That Sexual Harass...   
...              ...                                                ...   
50874           TECH    Google+ Now Open for Teens With Some Safeguards   
50875           TECH                                           Web Wars   
50876           TECH  First White House Chief Technology Officer, An...   
50877           TECH         Watch The Top 9 YouTube Videos Of The Week   
50878           TECH  RIM CEO Thorsten Heins' 'Significant' Plans Fo...   

                                                 authors  \
0                                          Andy McDonald   
1                                             Ron Dicker   
2                                             Ron Dicker   
3                                             Ron Dicker   
4                                             Ron Dicker   
...                                                  ...   
50874    Larry Magid, Contributor\nTechnology journalist   
50875  John Giacobbi, Contributor\nTales from the Int...   
50876                                                      
50877                                    Catharine Smith   
50878                                   Reuters, Reuters   

                                                    link  \
0      https://www.huffingtonpost.com/entry/will-smit...   
1      https://www.huffingtonpost.com/entry/hugh-gran...   
2      https://www.huffingtonpost.com/entry/jim-carre...   
3      https://www.huffingtonpost.com/entry/julianna-...   
4      https://www.huffingtonpost.com/entry/morgan-fr...   
...                                                  ...   
50874  https://www.huffingtonpost.com/entry/google-pl...   
50875  https://www.huffingtonpost.com/entry/congress-...   
50876  https://www.huffingtonpost.comhttp://www.engad...   
50877  https://www.huffingtonpost.com/entry/watch-top...   
50878  https://www.huffingtonpost.com/entry/rim-ceo-t...   

                                       short_description        date  
0                               Of course it has a song.  2018-05-26  
1      The actor and his longtime girlfriend Anna Ebe...  2018-05-26  
2      The actor gives Dems an ass-kicking for not fi...  2018-05-26  
3      The "Dietland" actress said using the bags is ...  2018-05-26  
4      "It is not right to equate horrific incidents ...  2018-05-26  
...                                                  ...         ...  
50874  For the most part, teens' experience on Google...  2012-01-28  
50875  These "Web Wars" threaten to rage on for some ...  2012-01-28  
50876  He was appointed with a fair bit of fanfare as...  2012-01-28  
50877  If you're looking to see the most popular YouT...  2012-01-28  
50878  Verizon Wireless and AT&T are already promotin...  2012-01-28  

[50879 rows x 6 columns]

In [91]:
#Función entrenamiento y retorna el Accurancy Score, Precision Score, Recall Score y F1 Score

def PIA(mss,label):
    X_train, X_test, y_train, y_test=train_test_split(mss, label, random_state=1)
    count_vector=CountVectorizer()
    training_data=count_vector.fit_transform(X_train)
    training_data = count_vector.fit_transform(X_train)
    testing_data = count_vector.transform(X_test)
    naive_bayes = MultinomialNB()
    naive_bayes.fit(training_data, y_train)
    predictions = naive_bayes.predict(testing_data)
    matConfu=confusion_matrix(y_test, predictions)  
    print(matConfu) 
    vec=[accuracy_score(y_test, predictions),precision_score(y_test, predictions,average='macro'),recall_score(y_test, predictions,average='macro'),f1_score(y_test, predictions,average='macro')]
    return vec
    #return metricasMatrizConfusion(matConfu)

#Fila Real Columna Predicción
def accurancy(mat):
    acc=0
    diagonal=0
    total=0 
    for i in range(len(mat)):
        for j in range((len(mat))):
            if i==j: 
                diagonal+=mat[i][j]
            total+=mat[i][j]
    acc=diagonal/total
    return acc

#Fila Real Columna Predicción. Total de verdadero la cat A sobre el total de reales de A
def recall(mat):    
    mat=np.array(mat)
    return (mat[0][0])/(mat.sum(axis=1)[0])
    

#Fila Real Columna Predicción. Total de verdadero la cat A sobre el total de prediciones de A
def precision(mat):
    mat=np.array(mat)
    return (mat[0][0])/(mat.sum(axis=0)[0])

def F1Score(r,p):
    return (2*r*p)/(r+p)

def metricasMatrizConfusion(mat):
    vec=[accurancy(mat),precision(mat),recall(mat),precision(mat)]
    return vec

In [92]:
print("Matriz de confusión headline/short_description/headline + short_description ")
vecHL=PIA(df["headline"],df["category"])
vecSH=PIA(df["short_description"],df["category"])
vecHLSH=PIA(df["short_description"]+df["headline"],df["category"])
newDf = pd.DataFrame({"head line":vecHL, "short_description":vecSH, "head line + short_description":vecHLSH },index=["Accurancy Score","Precision Score","Recall Score","F1 Score"])
newDf

Matriz de confusión headline/short_description/headline + short_description 
[[3636  374    5]
 [ 267 7912   12]
 [ 101  221  192]]
[[2491 1519    5]
 [ 512 7663   16]
 [ 135  289   90]]
[[3689  321    5]
 [ 258 7922   11]
 [ 104  207  203]]


head line  short_description  head line + short_description
Accurancy Score   0.922956           0.805346                       0.928774
Precision Score   0.918937           0.804577                       0.925032
Recall Score      0.748361           0.577020                       0.760302
F1 Score          0.795208           0.617410                       0.806899

En todas sus métricas usando para entrenamiento "short_description" es menor a "head line" y a
"head line + short_description". Y a su vez usando "head line + short_description" es mayor en todas sus metricas a "head line". 

Por lo tanto, entre más información se sabe acerca de la categorias "ENTERTAINMENT, POLITICS, y TECH" la predicción será mayor
